# 0. Install Packages

In [ ]:
!pip install transformers

# 1. Import Packages

 - 본 실습에 필요한 패키지들을 불러옵니다.

In [ ]:
from transformers import GPT2Model
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast
import torch
from torch.utils.data import Dataset, DataLoader
import urllib
import pandas as pd

# 2. KoGPT2 Tokenizer

 - 사전 학습된 KoGPT2 Tokenizer를 불러옵니다.

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>', padding_side='right') 
sample_text = "근육이 커지기 위해서는"

tokens = tokenizer.tokenize(sample_text)
token_ids = tokenizer.encode(sample_text)

print(f' Sentence: {sample_text}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

# 3. KoGPT2 Models

 - GPT2Model과 GPT2LMHeadModel을 불러옵니다.

## 3-1. GPT2Model

 - GPT2Model은 hidden state를 출력합니다.
 
 - 본 예제에서는 네 개의 토큰에 대한 768차원의 벡터가 도출됩니다.

In [ ]:
gpt2_model = GPT2Model.from_pretrained('skt/kogpt2-base-v2')
hidden_states = gpt2_model(torch.tensor([token_ids]))
last_hidden_state = hidden_states[0]
print(last_hidden_state.shape)

## 3-2. GPT2LMHeadModel

 - GPT2LMHead는 next word prediction을 출력합니다.
 
 - 본 예제에서는 네 개의 토큰에 대한 51200 차원의 단어 확률 분포가 도출됩니다.

In [ ]:
gpt2lm_model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
outputs = gpt2lm_model(torch.tensor([token_ids]))
next_word_predictions = outputs[0]
print(next_word_predictions.shape)

 - 단어 확률 분포에 대해 argmax를 취해 가장 높은 확률을 보이는 단어를 찾습니다.
 
 - 본 예제에서는 "무엇보다" 라는 단어가 가장 높은 확률을 나타냅니다.

In [ ]:
next_word_distribution = next_word_predictions[0, -1, :]
next_word_id = torch.argmax(next_word_distribution)
next_word = tokenizer.decode(next_word_id)

print(f'Next word: {next_word}')

# 4. Text Generation Examples (Pre-trained model)

 - 두 가지 Text Generation 방법을 실험해봅니다.

## 4-1. Greedy Search

 - Greedy Search는 가장 높은 확률의 단어를 Greedy하게 찾는 방식으로 텍스트를 생성합니다.

In [ ]:
gen_ids = gpt2lm_model.generate(torch.tensor([token_ids]),
                           max_length=127,
                           repetition_penalty=2.0,
                           )

generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

## 4-2. Beam Search

 - Beam Search는 매 step마다 num_beams 개 만큼의 Top word selection path를 찾습니다.


In [ ]:
gen_ids = gpt2lm_model.generate(torch.tensor([token_ids]),
                           max_length=127,
                           repetition_penalty=2.0,
                           num_beams=5, 
                           )

generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

# 5. Fine tunning (Naver Movie review)

 - 네이버 영화 리뷰데이터를 활용하여 모델을 Fine Tuning 합니다.

## 5-1. Get Datasets

 - github으로부터 네이버 영화 리뷰데이터를 요청하여 내 pc에 저장합니다.
 
 - 데이터의 크기가 너무 큰 관계로, 본 실험에서는 테스트 데이터 셋만을 활용하여 모델을 학습시킵니다. 

In [ ]:
def get_naver_review_examples():
    #urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
    urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

    #train_data = pd.read_table('ratings_train.txt')
    test_data = pd.read_table('ratings_test.txt')
    
    return test_data

In [ ]:
naver_data = get_naver_review_examples()

In [ ]:
naver_data

 - Dataset Loader를 정의합니다.

In [ ]:
class NaverReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          padding='max_length',
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True,
        )

        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(label, dtype=torch.long)
        }
    
    def __len__(self):
        return len(self.texts)


In [ ]:
dataset = NaverReviewDataset(naver_data['document'], naver_data['label'], tokenizer, 100)
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [40000, 5000, 5000])

In [ ]:
train_set[0]

In [ ]:
batch_size = 8

train_dataloader = DataLoader(train_set, batch_size=batch_size,
                        shuffle=True)

valid_dataloader = DataLoader(valid_set, batch_size=batch_size,
                        shuffle=True)

test_dataloader = DataLoader(test_set, batch_size=batch_size,
                        shuffle=True)

In [ ]:
sample_data = next(iter(test_dataloader))
sample_data

## 5-2. Model Settings

 - Model의 환경을 설정합니다.

In [ ]:
gpt2lm_model.train()

learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpt2lm_model.parameters(), lr=learning_rate)

device = 'cuda'

epochs = 10
count = 0

In [ ]:
sample_inputs = sample_data['input_ids'].to(device)
sample_outputs = gpt2lm_model(sample_inputs, labels=sample_inputs)

In [ ]:
sample_outputs['loss']

## 5-3. Model Training

 - Model의 학습을 시작합니다.

In [ ]:
tot_train_loss = 0.0
tot_valid_loss = 0.0
prev_valid_loss = 10000

print('KoGPT-2 Training Start!')

for epoch in range(epochs):
    for batch, train_data in enumerate(train_dataloader):
        # train data를 모델에 입력하여 출력 값을 얻습니다.
        gpt2lm_model.to(device)
        train_inputs = train_data['input_ids'].to(device)
        train_outputs = gpt2lm_model(train_inputs, labels=train_inputs) # train_outputs = (train_loss, train_logits, train_past_hidden_states)
        
        train_loss, _ = train_outputs[:2]
        
        # valid data를 모델에 입력하여 출력 값을 얻습니다.
            
        valid_data = next(iter(valid_dataloader))

        gpt2lm_model.to(device)
        valid_inputs = valid_data['input_ids'].to(device)     
        valid_outputs = gpt2lm_model(valid_inputs, labels=valid_inputs)
        
        valid_loss, _ = valid_outputs[:2]
        
        gpt2lm_model.to(device)
        
        # train loss를 토대로 모델을 학습합니다.
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        tot_train_loss += train_loss.item()
        tot_valid_loss += valid_loss.item()
               
        # 200 batch 마다 학습 상황을 화면에 출력합니다.
        if count % 200 == 0:
            cnt = ((count+1) * batch_size)
            current_train_loss = tot_train_loss / cnt
            current_valid_loss = tot_valid_loss / cnt
            
            print(f'epoch : %5d | batch : %5d | train_loss : %.5f | valid_loss : %.5f' %(epoch+1, batch+1, current_train_loss, current_valid_loss))
            
            tot_train_loss = 0.0
            tot_valid_loss = 0.0
            
            count = 0
            
            # 이전 valid_loss 보다 현재의 valid_loss가 더 낮을 경우, 모델을 저장합니다.
            if prev_valid_loss > current_valid_loss:
                prev_valid_loss = current_valid_loss
                torch.save(gpt2lm_model.state_dict(), f'./KoGPT-model.pth')
        
        count += 1

In [ ]:
train_set[5]

In [ ]:
kogpt_load_path = f"./KoGPT-model.pth"

gpt2lm_model.load_state_dict(torch.load(kogpt_load_path))

In [ ]:
gpt2lm_model.to(device)

sample_text = "정말 재미"

tokens = tokenizer.tokenize(sample_text)
token_ids = tokenizer.encode(sample_text)

gen_ids = gpt2lm_model.generate(torch.tensor([token_ids]).to(device),
                           max_length=127,
                           repetition_penalty=1.0,
                           num_beams=5)

generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

In [ ]:
import re

p = re.compile('<pad>')
re.sub(p, '', generated)

# 6. Fine Tuning 2 (Classification Task)

 - Dateset을 가져옵니다.

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>', padding_side='left') 

In [ ]:
batch_size = 16

naver_data = get_naver_review_examples()

dataset = NaverReviewDataset(naver_data['document'], naver_data['label'], tokenizer, 100)
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [40000, 5000, 5000])

train_dataloader = DataLoader(train_set, batch_size=batch_size,
                        shuffle=True)

valid_dataloader = DataLoader(valid_set, batch_size=batch_size,
                        shuffle=True)

test_dataloader = DataLoader(test_set, batch_size=batch_size,
                        shuffle=True)

 - GPT Classifier를 정의합니다.

In [ ]:
class GPT2SentimentClassifier(torch.nn.Module):

    def __init__(self, n_classes):
        super(GPT2SentimentClassifier, self).__init__()

        self.gpt_model = GPT2Model.from_pretrained('skt/kogpt2-base-v2')
        self.drop = torch.nn.Dropout(p=0.1)
        self.out = torch.nn.Linear(self.gpt_model.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        hidden_states = self.gpt_model(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        last_hidden_state = hidden_states[0]
        
        output = self.drop(last_hidden_state[:, -1, :])

        return self.out(output)


 - Model의 환경을 설정합니다.

In [ ]:
gpt_clf = GPT2SentimentClassifier(n_classes=1)
gpt_clf.train()

learning_rate = 5e-5
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(gpt_clf.parameters(), lr=learning_rate)

device = 'cuda'

epochs = 1
count = 0

 - 정확도 계산 함수를 정의합니다. 

In [ ]:
def cal_correct_num(predicts, labels):
    predicts_ = predicts >= 0.5
    correct_num = torch.sum(predicts_ == labels)
        
    return correct_num

 - Model의 학습을 시작합니다

In [ ]:
tot_train_loss = 0.0
tot_valid_loss = 0.0

train_correct_num = 0
valid_correct_num = 0

prev_valid_loss = 10000

print('KoGPT-2 Training Start!')

for epoch in range(epochs):
    for batch, train_data in enumerate(train_dataloader):
        # train data를 모델에 입력하여 출력 값을 얻습니다.
        gpt_clf.to(device)
        train_inputs = train_data['input_ids'].to(device)
        train_masks = train_data['attention_mask'].to(device)
        train_labels = train_data['labels'].to(device)
        
        train_outputs = gpt_clf(train_inputs, train_masks)
        
        train_loss = criterion(train_outputs.view(-1), train_labels.float())
        
        # valid data를 모델에 입력하여 출력 값을 얻습니다.
            
        valid_data = next(iter(valid_dataloader))

        gpt_clf.to(device)
        valid_inputs = valid_data['input_ids'].to(device)    
        valid_masks = valid_data['attention_mask'].to(device)
        valid_labels = valid_data['labels'].to(device)
        
        valid_outputs = gpt_clf(valid_inputs, valid_masks)
        
        valid_loss = criterion(valid_outputs.view(-1), valid_labels.float())
        
        gpt_clf.to(device)
        
        # train loss를 토대로 모델을 학습합니다.
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        tot_train_loss += train_loss.item()
        tot_valid_loss += valid_loss.item()
        
        train_correct_num += cal_correct_num(torch.sigmoid(train_outputs.view(-1)), train_labels.float())
        valid_correct_num += cal_correct_num(torch.sigmoid(valid_outputs.view(-1)), valid_labels.float())
               
        # 200 batch 마다 학습 상황을 화면에 출력합니다.
        if count % 200 == 0:
            cnt = ((count+1) * batch_size)
            current_train_loss = tot_train_loss / cnt
            current_valid_loss = tot_valid_loss / cnt
            
            train_acc = train_correct_num / cnt
            valid_acc = valid_correct_num / cnt
            
            print(f'epoch : %5d | batch : %5d | train_loss : %.5f | valid_loss : %.5f | train_acc : %.5f | valid_acc : %.5f' %(epoch+1, batch+1, current_train_loss, current_valid_loss, train_acc, valid_acc))
            
            tot_train_loss = 0.0
            tot_valid_loss = 0.0
            
            train_correct_num = 0
            valid_correct_num = 0
            
            count = 0
            
            # 이전 test_loss 보다 현재의 test_loss가 더 낮을 경우, 모델을 저장합니다.
            if prev_valid_loss > current_valid_loss:
                prev_valid_loss = current_valid_loss
                torch.save(gpt_clf.state_dict(), f'./KoGPT-Classifier-model.pth')
        
        count += 1